In [1]:
# add root to sys path
import os
import sys

current_dir = os.getcwd()
root = os.path.abspath(os.path.join(current_dir,".."))
if root not in sys.path:
    sys.path.append(root)

In [2]:
%load_ext autoreload
%autoreload 2

In [2]:
ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')
openai_api_key = os.getenv('OPENAI_API_KEY')
os.environ['ANTHROPIC_API_KEY'] = ANTHROPIC_API_KEY

In [ ]:
from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(model='claude-sonnet-4-20250514', temperature=0)  # Smart & expensive'''

answer = llm.invoke({'question':'hello'})
print(answer.content)

In [ ]:
test_state

In [4]:
# test in isolation the agent components step by step

import agent
from langchain_core.messages import AIMessage

graph = agent.graph
create_config = agent.create_config
objects_documentation = agent.objects_documentation
sql_dialect = agent.sql_dialect
orchestrator = agent.orchestrator
run_control_flow = agent.run_control_flow
generate_answer = agent.generate_answer

# Import initialization components
from src.init.initialization import (
    llm, llm_fast, create_config, tracer,
    objects_documentation, sql_dialect, connection_string
)

vector_store = None  # reset vector store
question = 'Which practice segment has the highest asset growth rate in the last 12 months?' 
test_state = {
'objects_documentation':objects_documentation,
'sql_dialect': sql_dialect,
'messages_log':[],
'intermediate_steps' : [],
'analytical_intent': [],
'current_question':question,
'current_sql_queries': [],
'generate_answer_details': {},
'llm_answer': AIMessage(content=''),
'scenario': '',
'search_terms_output':{}
}

#orchestrator.(test_state)
#test_state = run_control_flow(test_state) # extract_analytical_intent
#test_state = run_control_flow(test_state) # create sql query + execute sql query
#orchestrator(test_state)
#test_state = run_control_flow(test_state) # generate answer + manage memory
# test_state = generate_answer.invoke({'state':test_state})

In [5]:
test_state = orchestrator(test_state)
test_state = run_control_flow(test_state) # clarification_check
test_state = run_control_flow(test_state) # extract_analytical_intent
test_state = run_control_flow(test_state) # create sql query 
test_state = run_control_flow(test_state) # execute sql query 
test_state = run_control_flow(test_state) # generate answer 
test_state = run_control_flow(test_state) # add_assumptions
#test_state = run_control_flow(test_state) # manage_memory_chat_history

In [ ]:
run_control_flow(test_state)

In [ ]:
# test agent: Start a new conversation

######  start the imports

import agent
from langchain_core.messages import AIMessage

graph = agent.graph
create_config = agent.create_config
objects_documentation = agent.objects_documentation
sql_dialect = agent.sql_dialect

# Import initialization components
from src.init.initialization import (
    llm, llm_fast, create_config, tracer,
    objects_documentation, sql_dialect, connection_string
)

###### 

question = 'Which practice segment has the highest asset growth rate in the last 12 months?'
messages_log = []

initial_dict = {'objects_documentation':objects_documentation,
     'sql_dialect': sql_dialect,
     'messages_log': messages_log,
     'intermediate_steps':[],
     'analytical_intent': [],
     'current_question':question,
     'current_sql_queries': [],
     'generate_answer_details': {},
     'llm_answer': AIMessage(content=''),
     'scenario': '',
     'search_terms_output':{}
     }
     
vector_store = None  # reset vector store
config, thread_id = create_config('Run Agent',True)

result = graph.invoke(initial_dict, config = config)
display = f'''Analytical intent: {result['analytical_intent']}\n\nSQL query: {result['current_sql_queries']}\n\nGenerate Answer Details: {result['generate_answer_details']}\n\nAnswer: {result['llm_answer']}'''
print(display)

In [ ]:
# continue the conversation with the graph
question = 'first business line'

initial_dict['current_question'] = question
vector_store = None  # reset vector store
config, _ = create_config('generate_answer', False, thread_id) # (re-use same thread)
result = graph.invoke(initial_dict, config)
display = f'''Analytical intent: {result['analytical_intent']}\n\nSQL query: {result['current_sql_queries']}\n\nGenerate Answer Details: {result['generate_answer_details']}\n\nAnswer: {result['llm_answer']}'''
print(display)